# [YEAR] Barometric Pressure Data Preparation
ANALYST NAME | DATE

This notebook facilitates preparation of the barometric pressure time series used for compensating the unvented Levelogger time series.

When possible, the analyst will first inspect available Barologger time series. In order to use the Barologger time series for the compensation step, there should be no obvious erroneous segments in the time series and, for the sake of consistency when completing the subsequent Levelogger post-processing, the time series must span the Levelogger time series for all sites. That is, the first Barologger data point must be before the first Levelogger data point (across all sites) and the last Barologger data point must be after the last Levelogger data point (for all sites.

If no such Barologger time series exists, a different baromertric time series should be used. This notebook shows how to use NCAR reanalysis data to get a time series for surface pressure.

Once all steps have been completed, a barometric pressure time series will be ready to load for the compensation step.

## Import Relevant Libraries
**Analyst TODO**: Nothing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join('..', '..', 'src')))

import config
import level_baro_utils

sys.path.remove(os.path.abspath(os.path.join('..', '..', 'src')))

## Inspect Available Barologger Data
**Analyst TODO**: Inspect each available Barologger time series
* assign a string (format 'YYYY') representing the collection year of the data to the variable `collection_year`
* assign a string (format 'YY/YY') representing the span in years of the data to the variable `span`

Then, for each available time series
1. create a new Markdown cell, label ### Baro Sample X, where X is the sample number
2. make a copy of the code cell containing the site information and call to the `inspect_baro` function
3. assign a string representing the short name of the site to the variable `sitename_short`. Note, this is the name of the site contained in the filename (no spaces, may have abbreviations)
4. assign a string representing the long name of the site to the variable `long_short`. Note, this is the more "readable" name of the site (spaces, no abbreviations) used for plotting
5. Run the code cell, inspect the plot, and verify the time series spans all Levelogger time series. That is, the first Barologger data point must be before the first Levelogger data point (across all sites) and the last Barologger data point must be after the last Levelogger data point (for all sites.

If no valid Barologger time series exists, move on the next section: Prepare NCAR Reanalysis Pressure Data as Needed

In [ ]:
def inspect_baro(sitename_short, sitename_long, site_info):
    baro_path = os.path.join('..', 'data', 'normalized_raw', f'baro_{sitename_short}_{collection_year}.csv')

    baro_df = level_baro_utils.load_normalized_solinst_data(baro_path)
    level_baro_utils.plot_solinst_pressure_temp(baro_df, site_info, sensor_type='baro')

    start_date = baro_df.index[0]
    end_date = baro_df.index[-1]

    print(f'Time Series Start: {start_date}')
    print(f'Time Series End: {end_date}')
    
    return baro_df

In [ ]:
# TODO: specift collection year and span
# e.g.
# collection_year = '2019'
# span = '18/19'

collection_year = ''
span = ''

### Baro Sample 1

In [ ]:
# TODO: copy this code cell for each sample
# e.g.
# sitename_short = 'ConnessCrk'
# sitename_long = 'Conness Creek'

sitename_short = ''
sitename_long = ''
site_info = {'sitecode':-1,
             'site':sitename_long,
             'collection_year':collection_year,
             'span':span}

baro_sample1_df = inspect_baro(sitename_short, sitename_long, site_info)

## Prepare NCAR Reanalysis Pressure Data as Needed
**Analyst TODO**: 

For previous analysis, we used NCEP-NCAR Reanalysis data (https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html). If using another data source, the workflow provided below may need to be tweaked. 

Begin by downloading data for 3 variables: surface pressure, surface temperature, and geopotential height. Choose the time series with highest temporal resolution (for NCEP_NCAR Reanalysis, 4x daily). Since the levelogger data spans two calendar years, download the complete reanalysis time series for both years of interest. Create a new subdirectory in the /data directory called /ncar_reanalysis, and move the downloaded files to this directory.

### Load Data
**Analyst TODO**: Replace the bracketed segments of the paths with the correct file name.

In [ ]:
# surface pressure
p1_path = '../data/ncar_reanalysis/[PRESSURE_FILENAME]'
p2_path = '../data/ncar_reanalysis/[PRESSURE_FILENAME]'

# surface temperature (all lat/lon)
st1_path = '../data/ncar_reanalysis/[SURFACETEMP_FILENAME]'
st2_path = '../data/ncar_reanalysis/[SURFACETEMP_FILENAME]'

# height
hgt_path = '../data/ncar_reanalysis/[HEIGHT_FILENAME]'

In [ ]:
p1_global_ds = xr.open_dataset(p1_path)
p2_global_ds = xr.open_dataset(p2_path)

st1_global_ds = xr.open_dataset(st1_path)
st2_global_ds = xr.open_dataset(st2_path)

hgt_ds = xr.open_dataset(hgt_path)

### Isolate Desired Lat Long
**Analyst TODO**: Nothing

In [ ]:
p1 = p1_global_ds.pres.sel(lat=config.NCAR_TUM_LAT, lon=config.NCAR_TUM_LON)
p2 = p2_global_ds.pres.sel(lat=config.NCAR_TUM_LAT, lon=config.NCAR_TUM_LON)

st1 = st1_global_ds.air.sel(lat=config.NCAR_TUM_LAT, lon=config.NCAR_TUM_LON)
st2 = st2_global_ds.air.sel(lat=config.NCAR_TUM_LAT, lon=config.NCAR_TUM_LON)

hgt = hgt_ds.hgt.sel(lat=config.NCAR_TUM_LAT, lon=config.NCAR_TUM_LON)

### Merge Each Time Dependent Variable into Single Time Series
**Analyst TODO**: Inspect the side by side and merged time series plots. Verify everything looks reasonable.

#### Pressure

In [ ]:
p1.plot()
p2.plot()

In [ ]:
p1p2 = xr.merge([p1, p2])
p1p2.pres

In [ ]:
p1p2.pres.plot()
plt.ylabel('Pascals')

#### Air Temp

In [ ]:
st1.plot()
st2.plot()

In [ ]:
st1st2 = xr.merge([st1, st2])

In [ ]:
st1st2.air.plot()
plt.ylabel('K')

### Use Hypsometric Equation to Solve for Pressure at Tuolumne Elevation
Here we apply the hypsometric equation using the surface pressure, surface temperature, and geopotential height (the height of the surface) to compute the surface pressure for Tuolumne's elevation. Note that we approximate the virtual temperature with the actual temperature and the surface elevation with the geopotential height. We also assume a lapse rate of -6.5 C/km to compute the temperature in the middle of the column.

**Analyst TODO**: Nothing

In [ ]:
h_tum = config.TUM_ELEVATION  # h2, m
h_pixel = hgt.values[0]  # h1, m
h = h_tum - h_pixel  # h2 - h1, m
R = 287.052874  # specific gas constant for dry air, J K^-1 kg^-1
g = 9.80665  # m / s^2
P1 = p1p2
lapse_rate = -6.5 # C/km
T = st1st2.air + (lapse_rate * (h / 10**3) / 2)

P2 = P1 * np.exp((-1 * h * g) / (R * T))

P2

### Convert from PA to cm H2O
**Analyst TODO**: Nothing

In [ ]:
P2 *= level_baro_utils.PA_TO_CM

### Inspect
**Analyst TODO**: Inspect the output plot. The pressure should be on the order of 700-800 cm H2O

In [ ]:
P2.pres.plot()

### Compare to Barologger Data
**Analyst TODO**: Plot available any available Barologger data together with the reanalysis data. Verify the time series resemble one another. 

In [ ]:
P2.pres.plot()

# TODO: choose desired baro sample, if available
# e.g.
# baro_sample1_df['baro_cm'].plot()

### Save Data
**Analyst TODO**: Nothing

In [ ]:
P2.to_netcdf(f'../data/ncar_reanalysis/hypso_pres.sfc.{span}.nc')